In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import re
import tkinter as tk
from tkinter import messagebox
from transformers import pipeline

# -------------------------
# Config / thresholds
# -------------------------
TOX_THRESHOLD = 0.35         # direct toxicity threshold (tune)
NEG_SENT_THRESH = 0.60       # negative sentiment threshold
SARCASM_THRESH = 0.60        # sarcasm confidence threshold
THREAT_REGEX = re.compile(r"\b(i\s+will|i'll|i am going to|im gonna|i'm going to)\b.*\b(kill|hurt|destroy|shoot|stab|die|beat)\b", re.I)

# -------------------------
# Whitelist & special cases
# -------------------------
WHITELIST = {
    "allah u akbar",
    "allahu akbar",
    "om namah shivay",
    "hallelujah",
    # extend with common benign phrases you want to never flag
}

# Phrases we know are coded slurs or special toxic lexicalizations (domain specific)
SPECIAL_CASES = {
    "ola u uber": ["coded_religious_slur"],
    # add other exact normalized mappings as you discover them
}

# Polite phrases that often get mis-detected as sarcastic — suppress sarcasm tagging if exact match (or substring)
POLITE_PHRASES = [
    "have a nice day",
    "have a good day",
    "nice meeting you",
    "good luck",
    "thank you",
    "thanks",
    "have a great day",
    "take care",
]

# -------------------------
# Load models
# -------------------------
# Note: these model loads can be slow on first run.
toxicity_model = pipeline("text-classification", model="unitary/toxic-bert", top_k=None)
emotion_model  = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", top_k=None)
sentiment_model= pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# Sarcasm T5 model requires extra deps; if you had issues before you may switch to a different sarcasm model.
sarcasm_model  = pipeline("text-classification", model="mrm8488/t5-base-finetuned-sarcasm-twitter")

# -------------------------
# Helper: normalize
# -------------------------
def normalize_text(s: str) -> str:
    return re.sub(r'\s+', ' ', s.strip().lower())

# -------------------------
# Main analyze function
# -------------------------
def analyze_text(message,
                 tox_threshold=TOX_THRESHOLD,
                 neg_sent_thresh=NEG_SENT_THRESH,
                 sarcasm_thresh=SARCASM_THRESH):
    norm = normalize_text(message)

    # 0) Whitelist / exact safe phrase early exit
    if norm in WHITELIST:
        return {
            "input": message,
            "toxic_tags": [],
            "emotion": ("neutral", 1.0),
            "sentiment": {"label": "NEUTRAL", "score": 1.0},
            "raw_toxicity": {},
            "sarcasm": ("LABEL_0", 0.0),
            "special_flags": [],
            "notes": ["whitelisted exact phrase"]
        }

    notes = []
    special_flags = SPECIAL_CASES.get(norm, [])

    # 1) Toxicity model (multi-label)
    tox_preds = toxicity_model(message)[0]  # list of dicts
    raw_tox = {p['label']: p['score'] for p in tox_preds}
    direct_toxic = [lbl for lbl, s in raw_tox.items() if s > tox_threshold]

    # 1.1 Threat regex (explicit threat detection)
    threat_found = False
    if THREAT_REGEX.search(message):
        if "threat" not in direct_toxic:
            direct_toxic.append("threat")
        threat_found = True
        notes.append("threat_regex_matched")

    # 1.2 If special_cases matched, append those flags (these are domain-specific)
    if special_flags:
        direct_toxic.extend(special_flags)
        notes.append("matched_special_case")

    # 2) Sentiment
    sent = sentiment_model(message)[0]  # {'label': 'POSITIVE'/'NEGATIVE', 'score':...}

    # 3) Emotion (handle nested output)
    emo_raw = emotion_model(message)
    if isinstance(emo_raw[0], list):
        emo_raw = emo_raw[0]
    emotion_label = emo_raw[0]['label']
    emotion_score = emo_raw[0]['score']

    # 3.5 Polite-phrase suppression: if message contains a polite phrase substring, suppress sarcasm tagging
    polite_hit = any(phrase in norm for phrase in POLITE_PHRASES)
    if polite_hit:
        notes.append("polite_phrase_suppressed_sarcasm")

    # 4) Sarcasm model
    sarc_raw = sarcasm_model(message)[0]  # {'label': 'LABEL_0'/'LABEL_1', 'score':...}
    sarc_label = sarc_raw.get('label', 'LABEL_0')
    sarc_score = float(sarc_raw.get('score', 0.0))
    is_sarcastic = (sarc_label == "LABEL_1" and sarc_score >= sarcasm_thresh)

    # 5) Decide sarcasm-based tags (only if no direct toxicity and not a threat)
    sarcastic_praise = False
    sarcastic_insult = False
    if not direct_toxic and not threat_found and is_sarcastic and not polite_hit:
        # If sarcasm + positive sentiment + joyful emotion => sarcastic_praise (taunting praise)
        if sent['label'] == "POSITIVE" and emotion_label in ("joy", "surprise"):
            sarcastic_praise = True
            notes.append("sarcastic_praise_detected")
        # If sarcasm + negative sentiment or anger => sarcastic_insult
        elif sent['label'] == "NEGATIVE" or emotion_label == "anger":
            sarcastic_insult = True
            notes.append("sarcastic_insult_detected")
        # Otherwise, if sentiment neutral but emotion angry -> insult
        elif sent['label'] == "NEUTRAL" and emotion_label == "anger":
            sarcastic_insult = True
            notes.append("sarcastic_insult_from_anger")

    # 6) Negative sentiment catch-all (only if not direct toxic and not joyful)
    negative_sentiment_flag = False
    if not direct_toxic and not sarcastic_insult and not sarcastic_praise:
        if sent['label'] == "NEGATIVE" and sent['score'] > neg_sent_thresh and emotion_label != "joy":
            negative_sentiment_flag = True
            notes.append("negative_sentiment_detected")

    # 7) Combine final tags
    toxic_tags = list(dict.fromkeys(direct_toxic))  # deduplicate preserve order
    if sarcastic_praise:
        toxic_tags.append("sarcastic_praise")
    if sarcastic_insult:
        toxic_tags.append("sarcastic_insult")
    if negative_sentiment_flag:
        toxic_tags.append("negative_sentiment")

    # 8) Build result
    result = {
        "input": message,
        "toxic_tags": toxic_tags,
        "emotion": (emotion_label, emotion_score),
        "sentiment": sent,
        "raw_toxicity": raw_tox,
        "sarcasm": (sarc_label, sarc_score),
        "special_flags": special_flags,
        "notes": notes
    }
    return result

# -------------------------
# GUI
# -------------------------
def run_gui():
    root = tk.Tk()
    root.title("Enhanced Cyberbullying Detector — v2")
    root.geometry("680x640")
    root.resizable(False, False)

    frame_top = tk.Frame(root)
    frame_top.pack(pady=8)

    tk.Label(frame_top, text="Enter Message:", font=("Arial", 12)).grid(row=0, column=0, sticky="w")
    text_entry = tk.Text(frame_top, height=4, width=80, font=("Arial", 11))
    text_entry.grid(row=1, column=0, columnspan=4, padx=8, pady=6)

    # Buttons row
    def clear_input():
        text_entry.delete("1.0", "end")
        result_text.configure(state="normal")
        result_text.delete("1.0", "end")
        result_text.configure(state="disabled")

    clear_btn = tk.Button(frame_top, text="Clear", command=clear_input)
    clear_btn.grid(row=2, column=0, sticky="w", padx=(8,4), pady=6)

    auto_copy_var = tk.BooleanVar(value=False)
    auto_copy_chk = tk.Checkbutton(frame_top, text="Auto-copy results", variable=auto_copy_var)
    auto_copy_chk.grid(row=2, column=1, sticky="w", padx=4)

    copy_btn = tk.Button(frame_top, text="Copy to Clipboard", command=lambda: copy_to_clipboard(root, result_text))
    copy_btn.grid(row=2, column=2, sticky="w", padx=4)

    # Result display (selectable text)
    tk.Label(root, text="Analysis Result:", font=("Arial", 12)).pack(pady=(10,0))
    result_text = tk.Text(root, height=24, width=82, font=("Courier", 10), wrap="word")
    result_text.pack(padx=10, pady=6)
    result_text.configure(state="disabled")  # read-only until we write

    # helper
    def copy_to_clipboard(root_win, text_widget):
        content = text_widget.get("1.0", "end-1c")
        if content.strip():
            root_win.clipboard_clear()
            root_win.clipboard_append(content)
            messagebox.showinfo("Copied", "Results copied to clipboard.")
        else:
            messagebox.showinfo("Nothing", "No results to copy.")

    # Analyze action
    def on_analyze():
        msg = text_entry.get("1.0", "end").strip()
        if not msg:
            messagebox.showwarning("Input Required", "Please enter a message.")
            return

        # run analysis
        try:
            res = analyze_text(msg)
        except Exception as e:
            messagebox.showerror("Analysis error", str(e))
            return

        # Format output text
        out_lines = []
        out_lines.append("Input:")
        out_lines.append(res["input"])
        out_lines.append("")
        out_lines.append("Toxic tags: " + (", ".join(res["toxic_tags"]) or "None"))
        emo, emo_score = res["emotion"]
        out_lines.append(f"Emotion: {emo} ({emo_score:.2f})")
        sent = res["sentiment"]
        out_lines.append(f"Sentiment: {sent['label']} ({sent['score']:.2f})")
        sarc_label, sarc_score = res["sarcasm"]
        out_lines.append(f"Sarcasm: {sarc_label} (p={sarc_score:.2f})")
        if res["special_flags"]:
            out_lines.append("Special flags: " + ", ".join(res["special_flags"]))
        if res["notes"]:
            out_lines.append("Notes: " + "; ".join(res["notes"]))
        out_lines.append("")
        out_lines.append("Raw toxicity scores:")
        if res["raw_toxicity"]:
            out_lines.extend(f"{k}: {v:.2f}" for k, v in res["raw_toxicity"].items())
        else:
            out_lines.append("(none)")

        out_text = "\n".join(out_lines)

        # show in result box
        result_text.configure(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("1.0", out_text)
        result_text.configure(state="disabled")

        # auto-copy if requested
        if auto_copy_var.get():
            copy_to_clipboard(root, result_text)

    analyze_btn = tk.Button(root, text="Analyze", font=("Arial", 12), command=on_analyze)
    analyze_btn.pack(pady=(6,12))

    root.mainloop()

# run it
run_gui()


Device set to use mps:0
Device set to use mps:0
Device set to use mps:0
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-sarcasm-twitter and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use mps:0
